v1 à V2 -> on va deja voir si on prend un voxel dans la région 1 comme seed, ca marche sur les voxels de la region 1<br>
v2 à V3 -> on va  voir si on prend le signal moyen dans la région 1 comme seed, si ca marche sur les voxels de la region 1 <br>
v3 à v4 ->  on va  voir si on prend le signal moyen dans la région 1 comme seed, si ca marche sur les voxels de tout le cerveau <br>
v4 à v5 -> on dev la fct propre à intéger à aicha qui ne prend que 3 param= residus, moyenne des résidus sur une region, brain mask c1c2 normalizés <br>
v5 à v6 -> on va faire la correlation sur le signal sans arima

In [1]:
def computeCorrelations(residus, residusRegMean, brainMask):
    '''Function that takes 3 parameters
       residus: the residu of arima time courses
       residuRegMean: the residus mean on determined region i
       brainMask: the mask of gm and wm thresholded at 0,2
       Computes pearson correlations between the seed eg. residuRegMean
       and all the voxels of the brainMask
       Returns an Nifti images containing the correlations'''
    
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    import nibabel as nib
    from scipy.stats.stats import pearsonr
    
    # first we get the seed mean signal
    seed_ts_array = np.loadtxt(residusRegMean)
    
    # from an other hand we get the residus 4D matrix
    fmri_data=nib.load(residus) 
    fmri_array=np.asarray(fmri_data.dataobj)
    
    # we get the coordinnates of voxels in all gm and wm normalized todo 
    reg_data=nib.load(brainMask) 
    regarray=np.asarray(reg_data.dataobj)
    # transpose(nonzero(a))
    reg_coords = np.transpose(np.nonzero(regarray))
    volume_shape = reg_coords.shape
    print volume_shape
    coords = list(np.ndindex(volume_shape))
    print len(coords)
    
    # the we iterate the correlation calculation on all voxels of the brain mask

    # the correlation matrix is initialized with all values to 0 
    corr_matrix = np.full(reg_data.shape, 0, dtype=float)

    for i in range(reg_coords.shape[0]):
        target_array = fmri_array[reg_coords[i, 0], reg_coords[i, 1],reg_coords[i,2], :]
        #print target_array
        non_zero_nb = np.count_nonzero(target_array)
        
        # if target time courses are all null, we do not compute correlation
        if non_zero_nb:
            try:
                p = pearsonr(seed_ts_array,target_array) 
                if p[0] > 0.5:
                    print p[0]
                corr_matrix[reg_coords[i, 0], reg_coords[i, 1],reg_coords[i,2]] = p[0] 
            except:
                print "exception"
    
    # save matrix in a file
    # create the resulting image
    corr_image = nib.Nifti1Image(corr_matrix,affine=reg_data.affine, header=reg_data.header)
    # save the correlation array
    out_file = os.getcwd() + '/' + 'corr_regv6_bp.nii'
    nib.save(corr_image, out_file)
    
    

In [2]:
# correlations sans arima avec le signal bandpassed il faut auusi que j ai le mean du bandpassed sur region 1
computeCorrelations('/scratch/user/hirsch/datadir4/data_results_py/functionnal/bandpassedFile/wcrat0009_epi_s04_d0001_merged_bp.nii.gz',
                    '/scratch/user/hirsch/datadir4/data_results_py/functionnal/regMeants/_regMeants0/wcrat0009_epi_s04_d0001_merged_bp_ts.txt',
                    '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_c1c2_file/wc1t0009_t1_s03_maths_thresh.nii')

(193436, 3)
580308
0.536741991676
0.54934043465
0.505892936573
0.515664431814
0.608353900982
0.595157626907
0.624927916376
0.564724884551
0.544724325111
0.640409852421
0.626483386101
0.572091916786
0.57094748903
0.571010564582
0.538008023739
0.587634497168
0.552453479524
0.571884789596
0.60227418634
0.546349488104
0.505919035963
0.59197142012
0.518325556162
0.507418571613
0.518456914024
0.529022967346
0.502819153245
0.519794166254
0.506268790249
0.515112822283
0.500314005182
0.57322442438
0.58061670236
0.507182483165
0.552317449979
0.579610728737
0.511980042362
0.615942011332
0.533069839668
0.539515314248
0.544033918655
0.557735564437
0.512235605322
0.550237405776
0.573081620071
0.51457532007
0.501653220931
0.51153468651
0.561496087984
0.528134890356
0.515332383821
0.539135991263
0.539585583075
0.599409638796
0.501078057011
0.518434953927
0.502908305528
0.542701745387
0.560100702247
0.503119311414
0.561326248132
0.514819451378
0.520228692697
0.617559394775
0.578608679926
0.558062526974

In [ ]:
computeCorrelations('/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus/_report/arima_residu.nii.gz',
                    '/scratch/user/hirsch/datadir4/data_results_py/functionnal/regMeants/arima_residu_ts.txt',
                    '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_c1c2_file/wc1t0009_t1_s03_maths_thresh.nii')

In [2]:
# jusqu'à v4 on travaillait uniquement sur le gm
computeCorrelations('/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus/_report/arima_residu.nii.gz',
                    '/scratch/user/hirsch/datadir4/data_results_py/functionnal/regMeants/arima_residu_ts.txt',
                    '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files/wc1t0009_t1_s03.nii')

(197787, 3)
593361


## fin fonction et test

In [6]:
# first we get the region 1 image we will need it later for calculation in the same region but not in the general case 
# where we compute gm & wm
regSeedFile = '/homes_unix/hirsch/_ipnotebooks/region_image.nii'
reg_data=nib.load(regSeedFile) 

In [7]:
# then we get the seed mean signal
seedFile = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/regMeants/arima_residu_ts.txt'
seed_ts_array = np.loadtxt(seedFile)

In [8]:
# from an other hand we get the residus 4D matrix
fmri_file = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus/_report/arima_residu.nii.gz'
fmri_data=nib.load(fmri_file) 
fmri_array=np.asarray(fmri_data.dataobj)

In [10]:
# we get the coordinnates of voxels in all gm normalized todo get the wm also
regTargetFile = '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files/wc1t0009_t1_s03.nii'
reg_data=nib.load(regTargetFile) 
regarray=np.asarray(reg_data.dataobj)
# transpose(nonzero(a))
reg_coords = np.transpose(np.nonzero(regarray))
volume_shape = reg_coords.shape
print volume_shape
coords = list(np.ndindex(volume_shape))
print len(coords)

(197787, 3)
593361


In [11]:
# the we iterate the correlation calculation on all voxels of region 1

# np.full(shape, fill_value, dtype=None, order='C')
# the correlation matrix is initialized with all values to 0 
corr_matrix = np.full(reg_data.shape, 0, dtype=float)

for i in range(reg_coords.shape[0]):
    target_array = fmri_array[reg_coords[i, 0], reg_coords[i, 1],reg_coords[i,2], :]
    #print target_array
    non_zero_nb = np.count_nonzero(target_array)
    if non_zero_nb:
        try:
            p = pearsonr(seed_ts_array,target_array) 
            if p[0] > 0.5:
                print p[0]
            corr_matrix[reg_coords[i, 0], reg_coords[i, 1],reg_coords[i,2]] = p[0] 
        except:
            print "exception"
    
# save matrix in a file
# create the resulting image
corr_image = nib.Nifti1Image(corr_matrix,affine=reg_data.affine, header=reg_data.header)
# save the correlation array
out_file = os.getcwd() + '/' + 'corr_reg1v4.nii'
nib.save(corr_image, out_file)
